# SQL Data Query Language - One Page

This one pager is a reference for all SQL DQL (Data Query Language) commands.
Although some notes are added, this is intended to be a quick but comprehensive reference and reminder of all SQL query language commands.

**CONTENTS**

**1. SQL Query Order**

**2. Sorting**

**3. Limiting**

**3. Filtering**

**4. Joining Tables**

**5. Grouping**

# SQL Query Order

**SQL queries are evaluated in the following order:**

- FROM - choose and join tables
- WHERE - filters the data
- GROUP BY - aggregates the data
- HAVING - filters the aggregated data
- SELECT - returns the final data
- ORDER BY - sorts the final data
- LIMIT - limits sorted data to a row count

# Sorting - Order By

`
SELECT 
    first_name
FROM
    sales.customers
ORDER BY 
    first_name DESC
`

ORDER BY multiple cols - city first, then first name eg. 'New York, Brian', 'New York, Jane'

`
ORDER BY 
    city,
    first_name
`

ORDER BY expression

`
SELECT
    first_name
FROM
    sales.customers
ORDER BY 
    LEN(first_name) DESC
`

# Limiting - Offset and Fetch

OFFSET - means skip the first n rows

FETCH FIRST - means how many rows to fetch

- Often ORDER BY is used first, to put the data in it's correct order before limiting results.

`SELECT 
    team_name,
    points,
FROM 
    prem_table
ORDER BY 
    points DESC
OFFSET 4 ROWS
FETCH FIRST 13 ROWS`

# Limiting - Select Top

Get the Top n, top n%, or top nwith ties (means if tied for n number, show all tied results!)

Start the query with these options and continue with a normal query

`
SELECT TOP 10
SELECT TOP 1 PERCENT
SELECT TOP 3 WITH TIES
`

# Filtering - Distinct

`
SELECT DISTINCT 
    country
FROM 
    sales.customers
`

Use SELECT DISTINCT with multiple cols

`
SELECT DISTINCT
    city, state
FROM
    sales.customers
`

# Filtering - And Or

- AND is always evaluated first
- This query acts as: get brand_id = 1 OR (brand_id = 2 AND list_price > 40)

`
SELECT 
    product_name,
    brand_id,
    list_price
FROM 
    production.products
WHERE
    brand_id = 1
OR
    brand_id = 2
AND 
    list_price > 40
`

USE parenthesis to make sure you retrieve correct data

`
WHERE
    (brand_id 1 OR brand_id = 2)
AND
    list_price > 40
`

**Use the IN operator** - equivalent to many OR statements

`
SELECT 
    product_id,
    brand_id
FROM
    products
WHERE
    brand_id IN(1,2,3,4)
`

# Filtering - Between/ Not Between

`
SELECT
    product_id,
    product_name,
    list_price
FROM
    production.products
WHERE
    list_price NOT BETWEEN 149.99 AND 199.99
`

Between dates example

Use a string literal 'YYYYMMDD'

`
WHERE
    order_date BETWEEN '20170115' AND '20170117
`

# Filtering - Like

`
WHERE
    last_name LIKE 'z%'
`

`
WHERE
    last_name LIKE '%er'
`

`
WHERE
    last_name LIKE 't%s'
`

`
WHERE
    last_name LIKE '_u%'
`

Wilcard [] means either of these

`
WHERE
    last_name LIKE '[ZY]%'
`

[^A-X] means not starting with this range

`
WHERE
    last_name LIKE '[^A-X]%'
`

Escape clause allows us to use wilcard characters like '%'

`
WHERE 
    comment LIKE '%30!%%' ESCAPE '!';
`

# Filtering - Column Aliases

`
SELECT
    first_name + ' ' + last_name AS full_name
`

Use ' ' for spaces in alias

`
SELECT
    first_name + ' ' + last_name AS 'Full Name'
`

Use aliases for joining tables

`
SELECT
    c.customer_id,
    first_name,
FROM
    sales.customers c
INNER JOIN sales.orders o ON o.customer_id = c.customer_id
`

# Joining Tables - Inner Join

Find all in both tables

`
SELECT 
    c.id candidate_id,
    c.full_name candidate_name,
    e.id employeed_id,
    e.full_name employee_name
FROM 
    candidates c
    INNER JOIN 
        employees e
        ON c.id = e.id
`

Use joins to get information from other tables.

Often we want to retrieve a name instead of an id

`
SELECT 
    product_name, 
    category_name,
    list_price
FROM 
    production.products p
    INNER JOIN production.categories c
        ON p.category_id = c.category_id
`

# Joining Tables - Left Join

Return ALL candidates, not just matching ones in employees table -  NULL will appear if not in employee table

`
SELECT  
    c.id candidate_id,
    c.fullname candidate_name,
    e.id employee_id,
    e.fullname employee_name
FROM 
    hr.candidates c
    LEFT JOIN hr.employees e 
        ON e.fullname = c.fullname
`

List all product names with order_id (NULL will appear for any unsold products (products without order_id))

`
SELECT
    product_name,
    order_id
FROM
    production.products p
    LEFT JOIN sales.order_items o 
        ON o.product_id = p.product_id
    ORDER BY
        order_id;
`

# Joining Tables - Right Join

Opposite of left join

Show all employees in the right column, showing if they appear in the candidates table (or NULL) in the left column.

`
SELECT 
    c.id candidate_id,
    c.full_name candidate_name,
    e.id employee_id,
    e.full_name employee_name
FROM 
    candidates c
    RIGHT JOIN employees e
        ON c.full_name = e.full_name
`

# Joining Tables - Full Outer Join

Mix of left and right joins

Will show all canditates from left table, all employees from right and display if they appear in both tables or show NULL if they appear in one but not the other.

`
SELECT  
    c.id candidate_id,
    c.fullname candidate_name,
    e.id employee_id,
    e.fullname employee_name
FROM 
    hr.candidates c
    FULL JOIN hr.employees e 
        ON e.fullname = c.fullname;
`

RESULTS LIKE THIS:

1 John Doe 1 John Doe

2 Jane Smith NULL NULL

NULL NULL 2 Pete Jones

# Joining Tables - Cross Join

Called a CARTESIAN JOIN

Will create all possible combinations.

Often used as part of a query that will be used with a sub query

Store and Product match is a good example - will show all stores and product combinations...

`
SELECT
    s.store_id,
    p.product_id,
    ISNULL(sales, 0) sales
FROM
    sales.stores s
CROSS JOIN production.products p
    LEFT JOIN (....SUB QUERY...)
`

# Joining Tables - Self Join

Two versions of same table used (with aliases)

Often used for hierarchy queries such as employees and managers in the same table.

`
SELECT
    e.full_name employee,
    m.full_name manager
FROM 
    employees e
    LEFT JOIN employees m
    ON m.employee_id = e.manager_id
`

# Grouping - Group By

 COUNT num of orders per year per customer, eg. 1,2016,2...1,2017,1....2,2017,2...2,2018,1 etc.

`
SELECT
    customer_id,
    YEAR (order_date) order_year,
    COUNT (order_id) order_placed
FROM
    sales.orders
WHERE
    customer_id IN (1, 2)
GROUP BY
    customer_id,
    YEAR (order_date)
ORDER BY
    customer_id; 
`

COUNT customers in every city

`
SELECT
    city,
    COUNT (customer_id) customer_count
FROM
    sales.customers
GROUP BY
    city
ORDER BY
    city;
`

Get AVG list price (join used to retrieve brand name)

`
SELECT
    brand_name,
    AVG (list_price) avg_price
FROM
    production.products p
INNER JOIN production.brands b ON b.brand_id = p.brand_id
WHERE
    model_year = 2018
GROUP BY
    brand_name
ORDER BY
    brand_name;
`

Get net value of every order using SUM()

`
SELECT
    order_id,
    SUM (
        quantity * list_price * (1 - discount)
    ) net_value
FROM
    sales.order_items
GROUP BY
    order_id;
`

# Grouping - Having

HAVING is similar to WHERE clause, often used to filter GROUP BY

NOTE - HAVING is evaluated later in the query, so calculations not column aliases are used.


`
SELECT 
    salesperson_name,
    SUM(sales) sales_total
FROM 
    sales
GROUP BY 
    salesperson_name
HAVING 
    SUM(sales) < 2000
`

`
SELECT
    category_id,
    MAX (list_price) max_list_price,
    MIN (list_price) min_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    MAX (list_price) > 4000 OR MIN (list_price) < 500;
`

`
SELECT
    category_id,
    AVG (list_price) avg_list_price
FROM
    production.products
GROUP BY
    category_id
HAVING
    AVG (list_price) BETWEEN 500 AND 1000;
`

# Grouping - Grouping Sets

Without having to perform lengthy UNION statements

GROUPING SETS allow us to define multiple group by queries within the same query.

`
SELECT
	brand,
	category,
	SUM (sales) sales
FROM
	sales.sales_summary
GROUP BY
	GROUPING SETS (
		(brand, category),
		(brand),
		(category),
		()
	)
ORDER BY
	brand,
	category;
`

The result from the above query would return aggregated results for both brand and category, only brand results, only category results and neither brand or category results (total sales):

**Brand Category Sales**

NULL NULL 20000

NULL Bikes 10000

BrandA NULL 5000

BrandA Bikes 2000 


# Grouping - Cube

CUBE is a perfect follow on the GROUPING SETS () function

CUBE creates all possible grouping sets  from given columns

Ideal if you have more than 2 columns --- combos from 3 cols is large using GROUPING SET()...

`
SELECT
    d1,
    d2,
    d3,
    aggregate_function (c4)
FROM
    table_name
GROUP BY
    CUBE (d1, d2, d3);  
`

A partial cube is used to group by brand first, then group by category AND then by NULL showing all results.

**RESULTS:**

brand + category 

brand + NULL (total brand sales)

`
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    brand,
    CUBE(category);
`

# Grouping - Roll Up

Subset of GROUP BY - assumes hierarchy - creates less sets than CUBE

Often used in accounting totals, such as totals for year, month, quarter

ROLLUP(d1,d2,d3) produces:

(d1, d2, d3)
(d1, d2)
(d1)
()

`
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    ROLLUP(brand, category);
`

The above query returns:

brand + category

brand + NULL ( brand total)

NULL + NULL (overall total)

**RESULTS:** 

BrandA CategoryA 2000

BrandA NULL 4000

NULL NULL 6000


`
SELECT
    category,
    brand,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    ROLLUP (category, brand)
`

The above produces the following results:
    
category + brand

category + NULL (category total)

NULL + NULL (overall total)

`
SELECT
    brand,
    category,
    SUM (sales) sales
FROM
    sales.sales_summary
GROUP BY
    brand,
    ROLLUP (category);
`

The above is a 'partial' ROLLUP and will only return:

brand + category
brand + NULL (brand total sales)